In [3]:
from sklearn.decomposition import NMF
import pandas as pd
import numpy as np
from IPython.display import clear_output
import time

DATA_PATH = r'C:\Users\paul_\OneDrive\Pro\Advestis\K determination\analysis\datasets'

In [4]:
df = pd.read_csv(DATA_PATH + r'\swimmer\swimmer_2.csv')
m0 = df.values[:,2:].astype(np.float_)

# df = pd.read_csv(DATA_PATH + r'\sausage\Sausage Raw NIR.csv')
# m0 = df.values[:,8:].astype(np.float_)

# df = pd.read_csv(DATA_PATH + r'\esg\esg_posneg.csv')
# m0 = df.values[:,1:].astype(np.float_)

# df = pd.read_csv(DATA_PATH + r'\brunet\ALL-AML Brunet.csv')
# m0 = df.values[:,2:].astype(np.float_)

(n,p) = np.shape(m0)

# for Brunet only:
# m0 = np.log2(df.values[:,2:].astype(np.float_))
# m0-=np.repeat(np.min(m0, axis=0)[:,np.newaxis].T, n, axis=0)

min_comp = 10
max_comp = 20
n_runs = 50
iter_max = 10

In [5]:
time_start = time.time()
test_w = np.zeros(max_comp)
test_h = np.zeros(max_comp)

for n_comp in range(min_comp,max_comp+1):
    my_nmfmodel = NMF(n_components=n_comp, init='nndsvda', solver='cd', beta_loss='frobenius', max_iter=iter_max, random_state=0)
    #my_nmfmodel = NMF(n_components=n_comp, init='nndsvda', solver='mu', beta_loss='kullback-leibler', max_iter=200, random_state=0)
    w4 = my_nmfmodel.fit_transform(m0)
    h4 = my_nmfmodel.components_.T
    error = np.linalg.norm(m0 - w4 @ h4.T)
    #error = 1

    my_nmfmodel_random = NMF(n_components=n_comp, init='custom', solver='cd', beta_loss='frobenius', max_iter=iter_max, random_state=0)
    #my_nmfmodel_random = NMF(n_components=n_comp, init='custom', solver='mu', beta_loss='kullback-leibler', max_iter=200, random_state=0)
    np.random.seed(234)

    for i_run in range(0,n_runs):
        print('n_comp = ' + str(n_comp) + '; i_run = ' + str(i_run))
        w4_init = np.random.rand(n,n_comp); h4_init = np.random.rand(p,n_comp).T
        w4_random = my_nmfmodel_random.fit_transform(m0, W=w4_init.copy(), H=h4_init.copy())
        h4_random = my_nmfmodel_random.components_.T
        
        mcorr = np.corrcoef(w4, w4_random, rowvar=False)[0:n_comp,n_comp:2*n_comp]
        mcorr[np.isnan(mcorr)] = -1
        idx = mcorr.argmax(axis=1)
        mean_corr = np.mean([mcorr[i, idx[i]] for i in range(0,n_comp)])
        x = np.unique(idx).shape[0]
        # Move concordance into [0,1] range instead of [1/n_comp, 1] to cancel bias for low ranks
        # and correct for mean correlation level in matches
        test_w[n_comp-1] += ((x*(x-1)) / ((n_comp-1)*n_comp))*mean_corr
        # test_w[n_comp-1] += x / n_comp

        mcorr = np.corrcoef(h4, h4_random, rowvar=False)[0:n_comp,n_comp:2*n_comp]
        mcorr[np.isnan(mcorr)] = -1
        idx = mcorr.argmax(axis=1)
        mean_corr = np.mean([mcorr[i, idx[i]] for i in range(0,n_comp)])
        x = np.unique(idx).shape[0]
        test_h[n_comp-1] += ((x*(x-1)) / ((n_comp-1)*n_comp))*mean_corr
        # test_h[n_comp-1] += x / n_comp

    clear_output(wait=True)
    test_w[n_comp-1] /= (n_runs*error)
    test_h[n_comp-1] /= (n_runs*error)
    time_elapsed = (time.time() - time_start)
    print(time_elapsed)

51.43693733215332


In [6]:
# Cusum calculation
cusum = np.zeros(max_comp)
test = np.sqrt(test_w * test_h)
cusum[min_comp-2] = (test[min_comp-2]-test[min_comp-1] > 0)

for n_comp in range(min_comp,max_comp):
    if test[n_comp-1]-test[n_comp] > 0:
        deltax = 1
    else:
        deltax = -1
    
    cusum[n_comp-1] = max(cusum[n_comp-2]+deltax, 0)

cusum[max_comp-1] = cusum[max_comp-2]

# Rank estimation
n_comp_est = 999
for n_comp in range(1,max_comp-2):
    if cusum[n_comp-1]*cusum[n_comp]*cusum[n_comp+1]> 0:
        n_comp_est = n_comp
        break

print('Estimated rank = ', n_comp_est)

Estimated rank =  17


In [ ]:
# np.savetxt(DATA_PATH + r'\brunet\test_scipy_cd_brunet.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')
# np.savetxt(DATA_PATH + r'\sausage\test_scipy_cd_sausage.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')
np.savetxt(DATA_PATH + r'\swimmer\test_scipy_cd_swimmer.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')
# np.savetxt(DATA_PATH + r'\esg\test_scipy_cd_esg.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')


In [ ]:
cusum